In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install pycaret
from pycaret.classification import *
#from pycaret.regression import *

# Importing the data - only use *.MLReady.csv files!!!

In [26]:
df=pd.read_csv("D:\FM_Acq_Perf_FMAC_EPOCH2_ML.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,origIntRate,origUPB,origDate,firstPmtDate,origLTV,numBorrowers,...,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,1133339,1133339,100209956357,2,5.125,348000,2009-02-01,2009-04-01,87.0,1.0,...,5.04,5.13,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975
1,1133340,1133340,100392775700,3,4.625,195000,2009-02-01,2009-04-01,52.0,2.0,...,5.04,5.13,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441
2,1133341,1133341,100621428854,2,4.875,342000,2009-02-01,2009-04-01,80.0,1.0,...,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708
3,1133342,1133342,100720959726,1,5.375,93000,2009-02-01,2009-05-01,70.0,1.0,...,5.04,5.13,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758
4,1133343,1133343,100783704799,1,4.875,182000,2009-02-01,2009-04-01,76.0,2.0,...,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 60 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  119696 non-null  int64  
 1   Unnamed: 0.1                119696 non-null  int64  
 2   id                          119696 non-null  int64  
 3   origChannel                 119696 non-null  int64  
 4   origIntRate                 119696 non-null  float64
 5   origUPB                     119696 non-null  int64  
 6   origDate                    119696 non-null  object 
 7   firstPmtDate                119696 non-null  object 
 8   origLTV                     119696 non-null  float64
 9   numBorrowers                119696 non-null  float64
 10  origDebtIncRatio            119696 non-null  float64
 11  borrCreditScore             119696 non-null  float64
 12  loanPurp                    119696 non-null  int64  
 13  zipCode       

## Remove columns not involved in 'Foreclosure or Not' query

In [28]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','firstPmtDate','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
         'zeroBalDate','lastPdInstDate','forecloDate','dispDate','forecloCost','propRepCost','recovCosts',\
         'miscCost','holdTaxCost','saleProceed','credEnhProceed','repurchProceed','otherForecloProceed',\
         'nonIntUPB','prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator',\
         'deliqGood','deliqBad','deliqMax'],1,inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         119696 non-null  int64  
 1   origIntRate         119696 non-null  float64
 2   origUPB             119696 non-null  int64  
 3   origDate            119696 non-null  object 
 4   origLTV             119696 non-null  float64
 5   numBorrowers        119696 non-null  float64
 6   origDebtIncRatio    119696 non-null  float64
 7   borrCreditScore     119696 non-null  float64
 8   loanPurp            119696 non-null  int64  
 9   zipCode             119696 non-null  int64  
 10  pMIperct            119696 non-null  float64
 11  mortInsType         119696 non-null  float64
 12  bestCreditScore     119696 non-null  float64
 13  worstCreditScore    119696 non-null  float64
 14  avgCreditScore      119696 non-null  float64
 15  bankNumber          119696 non-nul

## Engineer originDate into Date features

In [30]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [31]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [32]:
df.drop(['origDate'],1,inplace=True)

In [33]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2009,2
1,2009,2
2,2009,2
3,2009,2
4,2009,2


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [34]:
df.zeroBalCode.unique()

array([ 9.,  1.,  3., 16.,  6.,  2., 15.])

In [35]:
df['zeroBalCode'] = df.zeroBalCode.replace(1,0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,16,6,2,15],1)

In [36]:
df.zeroBalCode.unique()

array([1., 0.])

In [37]:
#Checking datatypes of individual feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         119696 non-null  int64  
 1   origIntRate         119696 non-null  float64
 2   origUPB             119696 non-null  int64  
 3   origLTV             119696 non-null  float64
 4   numBorrowers        119696 non-null  float64
 5   origDebtIncRatio    119696 non-null  float64
 6   borrCreditScore     119696 non-null  float64
 7   loanPurp            119696 non-null  int64  
 8   zipCode             119696 non-null  int64  
 9   pMIperct            119696 non-null  float64
 10  mortInsType         119696 non-null  float64
 11  bestCreditScore     119696 non-null  float64
 12  worstCreditScore    119696 non-null  float64
 13  avgCreditScore      119696 non-null  float64
 14  bankNumber          119696 non-null  int64  
 15  stateNumber         119696 non-nul

# Review values for dates and slice (if needed) the timeframe you want

In [38]:
df.origYear.unique()

array([2009, 2010, 2011, 2012, 2013], dtype=int64)

In [39]:
df['origMonth'].unique()

array([ 2,  3,  5,  4,  6,  7,  8,  9, 10, 11, 12,  1], dtype=int64)

# Coerce the column dtype (int in this case)

In [40]:
df = df.astype({'origLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'borrCreditScore':'int','mortInsType':'int','bestCreditScore':'int',\
               'worstCreditScore':'int','avgCreditScore':'int'})
#,'zeroBalCode':'int'

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         119696 non-null  int64  
 1   origIntRate         119696 non-null  float64
 2   origUPB             119696 non-null  int64  
 3   origLTV             119696 non-null  int32  
 4   numBorrowers        119696 non-null  int32  
 5   origDebtIncRatio    119696 non-null  int32  
 6   borrCreditScore     119696 non-null  int32  
 7   loanPurp            119696 non-null  int64  
 8   zipCode             119696 non-null  int64  
 9   pMIperct            119696 non-null  float64
 10  mortInsType         119696 non-null  int32  
 11  bestCreditScore     119696 non-null  int32  
 12  worstCreditScore    119696 non-null  int32  
 13  avgCreditScore      119696 non-null  int32  
 14  bankNumber          119696 non-null  int64  
 15  stateNumber         119696 non-nul

In [42]:
#df.to_csv('D:\FM_FULL_EPOCH2_MLReady.csv')

# Review metrics last time before run (to have record of starting values)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   origChannel         119696 non-null  int64  
 1   origIntRate         119696 non-null  float64
 2   origUPB             119696 non-null  int64  
 3   origLTV             119696 non-null  int32  
 4   numBorrowers        119696 non-null  int32  
 5   origDebtIncRatio    119696 non-null  int32  
 6   borrCreditScore     119696 non-null  int32  
 7   loanPurp            119696 non-null  int64  
 8   zipCode             119696 non-null  int64  
 9   pMIperct            119696 non-null  float64
 10  mortInsType         119696 non-null  int32  
 11  bestCreditScore     119696 non-null  int32  
 12  worstCreditScore    119696 non-null  int32  
 13  avgCreditScore      119696 non-null  int32  
 14  bankNumber          119696 non-null  int64  
 15  stateNumber         119696 non-nul

In [44]:
df.head()

,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,...,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,2,5.125,348000,87,1,50,689,2,51,25.0,...,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975,2009,2
1,3,4.625,195000,52,2,54,703,1,82,0.0,...,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441,2009,2
2,2,4.875,342000,80,1,54,746,1,981,0.0,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2
3,1,5.375,93000,70,1,50,780,1,496,0.0,...,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758,2009,2
4,1,4.875,182000,76,2,22,802,1,18,0.0,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2


# Pycaret - Setup with categorical definition

### Use feature engineering, polynomial features, trig feats, PCA

In [45]:
# models=setup(df, target = 'rateDiffPct',
#              feature_interaction=True, feature_ratio=True,
#              polynomial_features=True, pca=True, pca_components=10,
#             numeric_features = ['origUPB','origLTV','numBorrowers','origDebtIncRatio',\
#                                'avgCreditScore'],
#             ignore_features = ['origIntRate','rank','deal'])

### Use feature engineering, polynomial features, trig feats

In [46]:
models=setup(df, target = 'zeroBalCode')

 
Setup Succesfully Completed!


,Description,Value
0,session_id,3155
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(119696, 31)"
4,Missing Values,False
5,Numeric Features,26
6,Categorical Features,3
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


## Compare Models with blacklist exclusions

In [47]:
model_results=compare_models(blacklist = ['tr','ransac']) #,'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

SystemExit: (Value Error): Estimator Not Available. Please see docstring for list of available estimators.

## Create models - store in variable

In [ ]:
catboost =create_model('catboost', fold = 10)

In [ ]:
catboost =create_model('catboost', fold = 50)

### Catboost fold results (not shown - reran in same cell)
* 10   0.5?
* 20   0.5633
* 50   0.5633

In [ ]:
lightgbm = create_model('lightgbm', fold=50)

### LightGBM fold results (not shown - reran in same cell)
    * 10   0.5?
    * 20   0.5?
    * 50   0.5585

In [ ]:
xgboost = create_model('xgboost', fold = 50)

## Tune Models

In [ ]:
tuned_catboost = tune_model('catboost') # from fold=50

* Made no difference creating the model with fold= 10 vs. fold = 50 so use fold = 10

In [ ]:
tuned_lightgbm = tune_model('lightgbm')

In [ ]:
tuned_xgboost = tune_model('xgboost')

## Ensemble models

In [ ]:
catboost_bagged = ensemble_model(catboost)

In [ ]:
catboost_boosted = ensemble_model(catboost)

## Blend 3 chosen Models

#### First blend 'non-tuned' models

In [ ]:
blend3 = blend_models(estimator_list = [catboost, lightgbm, xgboost])

#### Next blend tuned models

In [ ]:
blendTuned3 = blend_models(estimator_list = [tuned_catboost, tuned_lightgbm, tuned_xgboost])

## Evaluate Models

In [ ]:
plot_model(lightgbm, plot = 'residuals')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot='vc')

In [ ]:
plot_model(lightgbm, plot = 'feature')

In [ ]:
interpret_model(lightgbm, plot='correlation', feature='avgCreditScore')

In [ ]:
interpret_model(tuned_catboost)

In [ ]:
save_model()

In [ ]:
interpret_model(estimator=logreg_model, plot='reason')

In [ ]:
logreg_model2=create_model('xgboost')

In [ ]:
plot_model(estimator=logreg_model2, plot='feature')

In [ ]:
plot_model(estimator=logreg_model2)

In [ ]:
tunned_logreg_model=tune_model('catboost')

In [ ]:
interpret_model(estimator=tunned_logreg_model,plot='summary')

In [ ]:
plot_model(estimator=tunned_logreg_model,plot='feature')

In [ ]:
interpret_model(estimator=catboost, plot='correlation')

In [ ]:
interpret_model(estimator=catboost,plot='summary')